In [9]:
import pandas as pd
import numpy as np
import json
import nltk
from nltk.corpus import words, stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('words')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop = stopwords.words('english')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\magr3\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\magr3\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\magr3\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\magr3\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\magr3\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [1]:
from datasets import load_dataset
dataset = load_dataset('tiiuae/falcon-refinedweb', split='train', streaming=True)

c:\Users\magr3\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Resolving data files: 100%|██████████| 5534/5534 [00:01<00:00, 5424.36it/s]


In [2]:
import re
WORD1 = re.compile(r'[a-zA-Z]+', re.UNICODE)

In [6]:
def preprocces_text(df):
    df['content'] = ' '.join([word for word in WORD1.findall(df['content'].lower()) if word not in (stop) ]) #get rid of stop words
    #Lemmatizer
    w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    df['content'] = ' '.join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(df['content'])])
    return df

In [7]:
updated_dataset = dataset.map(preprocces_text)

In [10]:
print(next(iter(updated_dataset)))

{'content': 'birch found many place europe photo short trip baden baden cloud background messenger storm kyrill moment trip baden baden ast ray bilingual wordplay ast mean twig german baden baden sound like wordplay actual name rather well know spa town also date back roman time bad german word bath mirror effect turned nice like', 'url': 'http://100parts.wordpress.com/2012/08/04/astray-baden-baden-day-31/', 'timestamp': datetime.datetime(2013, 5, 18, 10, 42), 'dump': 'CC-MAIN-2013-20', 'segment': '1368696382261', 'image_urls': []}


In [11]:
import os
from tqdm import tqdm

total_size = 0  # Tamaño total acumulado en bytes
max_size_gb = 20  # Tamaño máximo en gigabytes

# Iniciar la barra de progreso
pbar = tqdm(total=max_size_gb*1024, unit='MB', desc='Progreso', unit_scale=True)

output_file = 'miniOscar.txt'

with open(output_file, 'w', encoding='utf-8') as f:
    for entry in updated_dataset:
        text = entry['text']
        text_size = len(text.encode('utf-8'))

        total_size += text_size
        pbar.update(total_size / (1024 * 1024) - pbar.n)  # Actualizar la barra de progreso

        if total_size / (1024 * 1024* 1024) >= max_size_gb:
            print("Se ha alcanzado el límite de 20 GB. Deteniendo la iteración.")
            break
        f.write(text + '\n')

pbar.close()  # Cerrar la barra de progreso al finalizar


Progreso:   0%|          | 30.5/20.5k [01:33<17:27:20, 3.07s/MB]
c:\Users\magr3\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\std.py:524: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
Progreso: 100%|██████████| 20.5k/20.5k [6:12:59<00:00, 1.09s/MB]

Se ha alcanzado el límite de 20 GB. Deteniendo la iteración.
